In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl

from pandas_datareader import data as pdr

import datetime as dt
import yfinance as yf

In [2]:
np.set_printoptions(precision = 3)

pd.set_option("display.float_format", lambda x: "%.3f" % x)

plt.style.use("ggplot")

mpl.rcParams["axes.grid"] = True
mpl.rcParams["grid.color"] = "grey"
mpl.rcParams["grid.alpha"] = 0.25

mpl.rcParams["axes.facecolor"] = "white"

mpl.rcParams["legend.fontsize"] = 14

In [3]:
from textblob import TextBlob

import spacy

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

import warnings

import csv

# https://spacy.io/usage/models

# import en_core_web_sm
# nlp = en_core_web_sm.load()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\fokta\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [4]:
data = pd.read_csv("filtered_data.csv")
data = data.drop(["Unnamed: 0"], axis = 1)
data["month"] = pd.to_datetime(data["dates"]).dt.month
data

,doc_ids,ids,company_ids,tickers,company_names,dates,year,month
0,10346313,10346313.F,4215,BME.L,B&M European Value Retail SA,17-Nov-15 8:30am GMT,2015,11
1,11171649,11171649.F,4454,BOO.L,Boohoo.Com PLC,29-Sep-15 7:30am GMT,2015,9
2,11171683,11171683.F,4454,BOO.L,Boohoo.Com PLC,6-May-15 8:00am GMT,2015,5
3,11189369,11189369.F,2922,ICA.ST,ICA Gruppen AB,11-Nov-15 9:00am GMT,2015,11
4,11189371,11189371.F,2922,ICA.ST,ICA Gruppen AB,19-Aug-15 8:00am GMT,2015,8
...,...,...,...,...,...,...,...,...
111727,12498685,12498685.F,7465,NOVN.VX,Novartis AG,23-Apr-19 10:59am GMT,2019,4
111728,12498736,12498736.F,2555,ONCE,Spark Therapeutics Inc,23-Apr-19 10:59am GMT,2019,4
111729,7634997,7634997.F,6466,JPM,JPMorgan Chase & Co,15-Jan-19 1:30pm GMT,2019,1
111730,7642483,7642483.F,6710,WFC,Wells Fargo & Co,15-Jan-19 3:00pm GMT,2019,1


In [5]:
spx_500 = pd.read_csv("SPX_500.csv")
spx = spx_500.drop(["Unnamed: 0"], axis = 1)
spx["month"] = pd.to_datetime(spx["dates"]).dt.month
spx

,ids,tickers,company_names,dates,year,month
0,5138263.F,LUV,Southwest Airlines Co,22-Jan-15 5:30pm GMT,2015,1
1,5436552.F,LUV,Southwest Airlines Co,23-Apr-15 4:30pm GMT,2015,4
2,5436557.F,LUV,Southwest Airlines Co,23-Jul-15 4:30pm GMT,2015,7
3,5436569.F,LUV,Southwest Airlines Co,22-Oct-15 4:30pm GMT,2015,10
4,5619337.F,LUV,Southwest Airlines Co,11-Feb-15 3:45pm GMT,2015,2
...,...,...,...,...,...,...
17452,12359836.F,TRMB,Trimble Inc,5-Mar-19 2:50pm GMT,2019,3
17453,12359845.F,TRMB,Trimble Inc,5-Mar-19 8:35pm GMT,2019,3
17454,12399723.F,TRMB,Trimble Inc,14-Mar-19 1:15pm GMT,2019,3
17455,11237358.F,TROW,T. Rowe Price Group Inc,21-Feb-18 7:00pm GMT,2018,2


In [6]:
com_scores_tf = pd.read_csv("firm_scores_TF.csv")
com_scores_tf = com_scores_tf.drop(["time"], axis = 1)
com_scores_tf["year"] = pd.to_datetime(data["dates"]).dt.year
com_scores_tf["month"] = pd.to_datetime(data["dates"]).dt.month
com_scores_tf

,innovation,integrity,quality,respect,teamwork,document_length,firm_id,year,month
0,1.567,0.404,1.972,0.455,0.202,1978,B&M European Value Retail SA,2015,11
1,4.824,0.062,1.546,0.000,0.309,1617,Boohoo.Com PLC,2015,9
2,2.730,0.072,2.083,0.359,0.287,1392,Boohoo.Com PLC,2015,5
3,1.426,0.317,1.347,0.079,0.159,1262,ICA Gruppen AB,2015,11
4,0.880,0.068,1.488,0.068,0.203,1478,ICA Gruppen AB,2015,8
...,...,...,...,...,...,...,...,...,...
111727,3.709,0.806,3.091,1.048,2.446,3721,Novartis AG,2019,4
111728,3.709,0.806,3.091,1.048,2.446,3721,Spark Therapeutics Inc,2019,4
111729,2.073,0.565,1.236,0.419,0.482,4775,JPMorgan Chase & Co,2019,1
111730,1.426,0.766,2.297,0.449,0.739,3787,Wells Fargo & Co,2019,1


In [7]:
scores_tf = pd.read_csv("scores_TF.csv")
scores_tf

,innovation,integrity,quality,respect,teamwork,document_length,Doc_ID
0,0,0,0,0,0,0,NaN
1,31,8,39,9,4,1978,10346313.000
2,78,1,25,0,5,1617,11171649.000
3,38,1,29,5,4,1392,11171683.000
4,18,4,17,1,2,1262,11189369.000
...,...,...,...,...,...,...,...
111728,138,30,115,39,91,3721,12498685.000
111729,138,30,115,39,91,3721,12498736.000
111730,99,27,59,20,23,4775,7634997.000
111731,54,29,87,17,28,3787,7642483.000


In [8]:
com_scores_tf.insert(0, "tickers", data["tickers"])

In [9]:
spx_scores = com_scores_tf[com_scores_tf["firm_id"].isin(spx["company_names"])]
spx_scores

,tickers,innovation,integrity,quality,respect,teamwork,document_length,firm_id,year,month
6,LUV,2.127,0.577,1.398,0.274,0.365,3291,Southwest Airlines Co,2015,1
15,TAP,3.072,0.212,1.430,0.583,0.318,1888,Molson Coors Brewing Co,2015,2
18,TSN,2.595,0.275,2.259,0.092,0.153,3276,Tyson Foods Inc,2015,1
19,TSN,2.861,0.263,2.160,0.088,0.409,3426,Tyson Foods Inc,2015,5
20,TSN,2.383,0.147,2.677,0.029,0.176,3399,Tyson Foods Inc,2015,8
...,...,...,...,...,...,...,...,...,...,...
111697,ON,1.560,0.246,1.724,0.328,0.328,1218,ON Semiconductor Corp,2019,4
111704,PFE,3.709,0.806,3.091,1.048,2.446,3721,Pfizer Inc,2019,4
111705,O,1.490,0.549,1.333,0.431,1.490,2550,Realty Income Corp,2019,4
111729,JPM,2.073,0.565,1.236,0.419,0.482,4775,JPMorgan Chase & Co,2019,1


In [10]:
grouping = (spx_scores.groupby(["tickers", "year"]).mean()).drop(["document_length", "month"], axis = 1).sum(axis = 1)
grouping

tickers  year
A        2015   6.332
         2016   7.058
         2017   7.878
         2018   6.686
         2019   9.232
                 ... 
ZTS      2015   9.606
         2016   7.732
         2017   8.651
         2018   8.013
         2019   7.990
Length: 2265, dtype: float64

In [11]:
data_2015 = grouping.unstack().iloc[ : , :1].sort_values(by =  2015, ascending = False).dropna()
data_2016 = grouping.unstack().loc[ : , [2016]].sort_values(by =  2016, ascending = False).dropna()
data_2017 = grouping.unstack().loc[ : , [2017]].sort_values(by =  2017, ascending = False).dropna()
data_2018 = grouping.unstack().loc[ : , [2018]].sort_values(by =  2018, ascending = False).dropna()
data_2019 = grouping.unstack().loc[ : , [2019]].sort_values(by =  2019, ascending = False).dropna()

In [12]:
data_2015_top = data_2015.iloc[0:30].index.to_list()
data_2015_bot = data_2015.iloc[-30: ].index.to_list()
data_2016_top = data_2016.iloc[0:30].index.to_list()
data_2016_bot = data_2016.iloc[-30: ].index.to_list()
data_2017_top = data_2017.iloc[0:30].index.to_list()
data_2017_bot = data_2017.iloc[-30: ].index.to_list()
data_2018_top = data_2018.iloc[0:30].index.to_list()
data_2018_bot = data_2018.iloc[-30: ].index.to_list()
data_2019_top = data_2019.iloc[0:30].index.to_list()
data_2019_bot = data_2019.iloc[-30: ].index.to_list()

In [13]:
ticker_name = pd.DataFrame([data_2015_top,data_2015_bot, data_2016_top,data_2016_bot,data_2017_top,data_2017_bot,data_2018_top, data_2018_bot,data_2019_top,data_2019_bot]).T
ticker_name.columns = ["top_2015","bot_2015","top_2016","bot_2016","top_2017","bot_2017","top_2018","bot_2018","top_2019","bot_2019"]
ticker_name

,top_2015,bot_2015,top_2016,bot_2016,top_2017,bot_2017,top_2018,bot_2018,top_2019,bot_2019
0,CRM,NEE,CRM,BIO,NKE,STE,CRM,EXR,GOOGL,APA
1,TGT,VRTX,CTSH,VLO,ATVI,MAA,ADBE,MLM,CRM,LNT
2,JCI,RHI,TGT,LYB,CRM,TRGP,NKE,PPL,CDW,FRC
3,CDNS,ES,CSCO,AOS,CTSH,PCAR,ANSS,MOS,NOW,RE
4,ADBE,PPL,ATVI,WEC,ADBE,KMI,CTSH,PPG,NDAQ,OKE
5,NKE,CZR,JCI,PXD,TGT,PPL,CSCO,VLO,ATVI,HIG
6,CSCO,LNT,IPG,ON,BA,AOS,MSFT,APA,NKE,DUK
7,CTSH,HES,WBA,PSX,CSCO,RHI,NOW,VRSN,IPG,MLM
8,CVS,VLO,ADBE,MAA,NOW,WEC,GOOGL,DVA,ADBE,MPC
9,MCD,COO,GWW,TER,IPG,APA,CDNS,D,CTSH,POOL


In [14]:
market_sentiment = pd.read_csv("pca_pls.csv")
market_sentiment["date"] = pd.to_datetime(market_sentiment["Unnamed: 0"], format = "%Y%m")
market_sentiment = market_sentiment.drop(["Unnamed: 0"], axis = 1)
market_sentiment

,pca,pls,date
0,-2.533,-0.450,1965-09-01
1,-2.317,-0.467,1965-10-01
2,-1.810,-0.158,1965-11-01
3,-1.853,-0.180,1965-12-01
4,-2.154,-0.312,1966-01-01
...,...,...,...
650,3.082,2.488,2022-02-01
651,1.588,0.286,2022-03-01
652,1.811,3.471,2022-04-01
653,-1.136,0.037,2022-05-01


In [15]:
pos = pd.read_csv(r"Pos_News.csv", sep = "|")
pos.dropna(inplace = True)
pos

,ticker,headlines,date,Event_Return_1,Event_Return_2,Event_Return_3,Event_Return_4,Event_Return_5,Event_Return_6,Excess_Return_RF,Return,Close,sent_lex
0,HIMX,Citing growing LCoS microdisplay activity amon...,2015-01-07,0.044,-0.011,0.066,0.025,0.000,-0.008,0.047,0.047,6.157,0.931
1,MSCI,MSCI enjoyed a late-day pop after activist inv...,2015-01-07,0.082,0.092,0.053,0.052,0.042,0.048,0.023,0.023,45.588,0.971
2,AFSI,The offering includes an underwriter option to...,2015-01-07,-0.027,-0.111,-0.023,0.009,-0.020,-0.040,0.014,0.014,24.282,0.537
3,MPW,"In addition to the 30M share sale, the underwr...",2015-01-07,0.025,0.073,0.033,-0.009,0.014,0.033,-0.001,-0.001,8.470,0.889
4,ZUMZ,Zumiez (NASDAQ:ZUMZ) reports comparable-store ...,2015-01-07,0.063,0.050,0.075,0.035,0.054,0.043,0.047,0.047,40.280,0.588
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20066,GBTC,Technicians were looking for another leg down ...,2019-12-23,0.049,0.071,-0.003,0.066,0.068,0.119,0.013,0.014,9.000,0.575
20067,ROK,JPMorgan has downgraded Rockwell Automation (N...,2019-12-23,-0.000,-0.004,-0.008,0.007,0.002,0.003,-0.000,-0.000,193.717,0.654
20068,SRPT,Sarepta Therapeutics (NASDAQ:SRPT) is up,2019-12-23,0.072,-0.010,0.065,0.081,0.071,0.034,0.075,0.075,135.580,0.150
20069,BAYRY,Bayer (OTCPK:BAYRY) started the week up 3.5% t...,2019-12-23,0.035,0.044,0.038,0.032,0.036,0.043,0.035,0.035,18.218,0.770


In [16]:
spx_sentiment = pos[pos["ticker"].isin(spx["tickers"])]
spx_sentiment["year"] = pd.to_datetime(spx_sentiment['date']).dt.year
spx_sentiment["month"] = pd.to_datetime(spx_sentiment['date']).dt.month

C:\Users\fokta\AppData\Local\Temp\ipykernel_8180\2714423772.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spx_sentiment["year"] = pd.to_datetime(spx_sentiment['date']).dt.year
C:\Users\fokta\AppData\Local\Temp\ipykernel_8180\2714423772.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spx_sentiment["month"] = pd.to_datetime(spx_sentiment['date']).dt.month


In [17]:
spx_sentiment_high = spx_sentiment[spx_sentiment["sent_lex"] > 0.7]

In [18]:
new_date = []

for i in spx_sentiment_high["date"]:
    new_date.append(i[ :-3])

In [19]:
spx_sentiment_high_copy = spx_sentiment_high.copy()
spx_sentiment_high_copy.insert(3, "dummy_date", pd.to_datetime(new_date, format = "%Y-%m"))
spx_sentiment_high_copy.reset_index(drop = True, inplace = True)

In [20]:
combined_data = pd.merge(spx_sentiment_high_copy, 
                  market_sentiment,
                 left_on = 'dummy_date',   
                 right_on = 'date'
                 )
combined_data = combined_data.drop(["date_y","dummy_date"], axis = 1)
combined_data.rename(columns = {"date_x" : "date"}, inplace = True)
combined_data.rename(columns = {"year_y" : "year"}, inplace = True)

In [21]:
combined_data_pos_pls = combined_data[combined_data["pls"] > 0]
combined_data_pos_pls

,ticker,headlines,date,Event_Return_1,Event_Return_2,Event_Return_3,Event_Return_4,Event_Return_5,Event_Return_6,Excess_Return_RF,Return,Close,sent_lex,year,month,pca,pls
0,MSCI,MSCI enjoyed a late-day pop after activist inv...,2015-01-07,0.082,0.092,0.053,0.052,0.042,0.048,0.023,0.023,45.588,0.971,2015,1,-0.506,1.006
1,JNPR,"Juniper (JNPR +3%), Sonus (SONS +3.2%) and Inf...",2015-01-07,0.037,0.007,0.051,0.008,-0.003,0.003,0.022,0.022,18.456,0.964,2015,1,-0.506,1.006
2,ENPH,Goldman has launched coverage on Enphase (NASD...,2015-01-07,-0.158,-0.257,-0.090,-0.187,-0.210,-0.216,-0.119,-0.119,11.400,0.917,2015,1,-0.506,1.006
3,TMUS,T-Mobile (TMUS +1.6%) added 1.28M branded post...,2015-01-07,0.076,0.085,0.079,0.049,0.036,0.049,0.051,0.051,28.250,0.961,2015,1,-0.506,1.006
4,PGR,Upgrading Progressive (NYSE:PGR) to Buy from N...,2015-01-07,0.029,-0.016,0.041,0.008,-0.003,-0.001,0.021,0.021,21.183,0.834,2015,1,-0.506,1.006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3491,LYB,LyondellBasell (LYB -1.7%) is lower after RBC ...,2019-12-20,-0.007,0.010,-0.009,-0.013,-0.010,0.003,-0.015,-0.015,77.309,0.961,2019,12,0.430,1.615
3492,CCL,Carnival (CCL +9.2%) rallies after easily topp...,2019-12-20,0.099,0.091,0.097,0.078,0.064,0.069,0.076,0.076,49.631,0.960,2019,12,0.430,1.615
3493,K,Bank of America Merrill Lynch lifts Kellogg (N...,2019-12-20,0.034,0.035,0.030,0.032,0.038,0.035,0.028,0.028,61.736,0.929,2019,12,0.430,1.615
3494,KMX,CarMax (NYSE:KMX) reports total used unit sale...,2019-12-20,-0.098,-0.094,-0.100,-0.060,-0.063,-0.054,-0.062,-0.062,92.710,0.970,2019,12,0.430,1.615


In [22]:
dummy_2016 = combined_data_pos_pls[combined_data_pos_pls["ticker"].isin(ticker_name["top_2015"])]
bool_dummy_2016 = dummy_2016[dummy_2016["year"] < 2017]
data_2016_returns = bool_dummy_2016[bool_dummy_2016["year"] > 2015]
dummy_2017 = combined_data_pos_pls[combined_data_pos_pls["ticker"].isin(ticker_name["top_2016"])]
bool_dummy_2017 = dummy_2017[dummy_2017["year"] < 2018]
data_2017_returns = bool_dummy_2017[bool_dummy_2017["year"] > 2016]
dummy_2018 = combined_data_pos_pls[combined_data_pos_pls["ticker"].isin(ticker_name["top_2017"])]
bool_dummy_2018 = dummy_2018[dummy_2018["year"] < 2019]
data_2018_returns = bool_dummy_2018[bool_dummy_2018["year"] > 2017]
dummy_2019 = combined_data_pos_pls[combined_data_pos_pls["ticker"].isin(ticker_name["top_2018"])]
bool_dummy_2019 = dummy_2019[dummy_2019["year"] < 2020]
data_2019_returns = bool_dummy_2019[bool_dummy_2019["year"] > 2018]

# Determining holding period

In [23]:
pos_news_df = pd.read_csv(r"positive (2).csv", sep = "|")
pos_news_df.dropna(inplace = True)

In [24]:
pos_spx = pos_news_df[pos_news_df["ticker"].isin(spx["tickers"])]
pos_spx_high = pos_spx[pos_spx["sent_lex"] > 0.7].iloc[ : , 6:-1]

In [25]:
pct_change_in_event_return = pd.DataFrame(pos_spx_high.T.diff().mean(axis =1))*100
pct_change_in_event_return.columns = ["pct_change_in_event_return"]
pct_change_in_event_return

,pct_change_in_event_return
Event_Return_4,NaN
Event_Return_5,0.045
Event_Return_6,0.137
Event_Return_7,0.104


# Top 30 firms

In [26]:
def getting_data(df, start, end): 
    df = pdr.get_data_yahoo(df,
                  start = start,
                   end = end)
    return df

In [27]:
data_top_2015 = getting_data(ticker_name["top_2015"],dt.datetime(2015,1,1),dt.datetime(2019,12,29))["Close"]
data_top_2016 = getting_data(ticker_name["top_2016"],dt.datetime(2015,1,1),dt.datetime(2019,12,29))["Close"]
data_top_2017 = getting_data(ticker_name["top_2017"],dt.datetime(2015,1,1),dt.datetime(2019,12,29))["Close"]
data_top_2018 = getting_data(ticker_name["top_2018"],dt.datetime(2015,1,1),dt.datetime(2019,12,29))["Close"]

In [28]:
data_top_2015.index = data_top_2015.index.strftime("%Y-%m-%d")
data_top_2016.index = data_top_2016.index.strftime("%Y-%m-%d")
data_top_2017.index = data_top_2017.index.strftime("%Y-%m-%d")
data_top_2018.index = data_top_2018.index.strftime("%Y-%m-%d")

In [29]:
first_2015 = data_top_2015.T[data_top_2015.T.index.isin(data_top_2015.T.index[data_top_2015.T.index.isin(data_2016_returns["ticker"])])]
first_2016 = data_top_2016.T[data_top_2016.T.index.isin(data_top_2016.T.index[data_top_2016.T.index.isin(data_2017_returns["ticker"])])]
first_2017 = data_top_2017.T[data_top_2017.T.index.isin(data_top_2017.T.index[data_top_2017.T.index.isin(data_2018_returns["ticker"])])]
first_2018 = data_top_2018.T[data_top_2018.T.index.isin(data_top_2018.T.index[data_top_2018.T.index.isin(data_2019_returns["ticker"])])]

In [30]:
def event_return(dataframe, shift_value, main_dataframe,groupby_dataframe):
    returns = np.log(dataframe / dataframe.shift(1))
    returns_next_day = returns.shift(shift_value)
    first_filtering = returns_next_day[returns_next_day.index.isin(main_dataframe)]
    second_filtering = first_filtering.unstack()[first_filtering.unstack().index.isin(groupby_dataframe)]
    making_list = (second_filtering.sort_index().values.tolist())
    return making_list

In [31]:
shift_value =  [-1,-4]
return_list_2016 = []
return_list_2017 = []
return_list_2018 = []
return_list_2019 = []


for i in shift_value:
    calculation_2016 = event_return(first_2015.T, i, data_2016_returns["date"], data_2016_returns.groupby(["ticker","date"]).sum().index)
    calculation_2017 = event_return(first_2016.T, i, data_2017_returns["date"], data_2017_returns.groupby(["ticker","date"]).sum().index)
    calculation_2018 = event_return(first_2017.T, i, data_2018_returns["date"], data_2018_returns.groupby(["ticker","date"]).sum().index)
    calculation_2019 = event_return(first_2018.T, i, data_2019_returns["date"], data_2019_returns.groupby(["ticker","date"]).sum().index)
    return_list_2016.append(calculation_2016)
    return_list_2017.append(calculation_2017)
    return_list_2018.append(calculation_2018)
    return_list_2019.append(calculation_2019)

In [58]:
top_strategy = pd.concat([(pd.Series(return_list_2016[0]) * 1 + pd.Series(return_list_2016[1]) * -1),\
          (pd.Series(return_list_2017[0]) * 1 + pd.Series(return_list_2017[1]) * -1),\
          (pd.Series(return_list_2018[0]) * 1 + pd.Series(return_list_2018[1]) * -1),\
          (pd.Series(return_list_2019[0]) * 1 + pd.Series(return_list_2019[1]) * -1)],axis =1)
top_strategy.columns = ["2016","2017","2018","2019"]

In [33]:
print(top_strategy.cumsum().apply(np.exp)["2016"].dropna().iloc[-1] - 1)
print(top_strategy.cumsum().apply(np.exp)["2017"].dropna().iloc[-1] - 1)
print(top_strategy.cumsum().apply(np.exp)["2018"].dropna().iloc[-1] - 1)
print(top_strategy.cumsum().apply(np.exp)["2019"].dropna().iloc[-1] - 1)

-0.0030904175880465123
0.1975343423114133
-0.018259817321523708
0.12655137527016214


In [34]:
top_cumsum = pd.concat([pd.Series(top_strategy.cumsum().apply(np.exp)["2016"].dropna().iloc[-1]),\
pd.Series(top_strategy.cumsum().apply(np.exp)["2017"].dropna().iloc[-1]),\
pd.Series(top_strategy.cumsum().apply(np.exp)["2018"].dropna().iloc[-1]),\
pd.Series(top_strategy.cumsum().apply(np.exp)["2019"].dropna().iloc[-1])],axis = 1)
top_cumsum.columns = ["2016","2017","2018","2019"]

In [35]:
def sharpe(data):
    sharpe_ratio = (data.mean()/data.std()) *252**0.5
    return sharpe_ratio

In [36]:
top_sharpe = pd.concat([pd.Series(sharpe(top_strategy["2016"])),pd.Series(sharpe(top_strategy["2017"])),\
                          pd.Series(sharpe(top_strategy["2018"])),pd.Series(sharpe(top_strategy["2019"]))],axis = 1)
top_sharpe.columns = ["2016","2017","2018","2019"]
top_sharpe

,2016,2017,2018,2019
0,-0.033,1.479,-0.117,0.682


# Bottom 30 firms

In [37]:
data_bot_2015 = getting_data(ticker_name["bot_2015"],dt.datetime(2015,1,1),dt.datetime(2019,12,29))["Close"]
data_bot_2016 = getting_data(ticker_name["bot_2016"],dt.datetime(2015,1,1),dt.datetime(2019,12,29))["Close"]
data_bot_2017 = getting_data(ticker_name["bot_2017"],dt.datetime(2015,1,1),dt.datetime(2019,12,29))["Close"]
data_bot_2018 = getting_data(ticker_name["bot_2018"],dt.datetime(2015,1,1),dt.datetime(2019,12,29))["Close"]

In [38]:
data_bot_2015.index = data_bot_2015.index.strftime("%Y-%m-%d")
data_bot_2016.index = data_bot_2016.index.strftime("%Y-%m-%d")
data_bot_2017.index = data_bot_2017.index.strftime("%Y-%m-%d")
data_bot_2018.index = data_bot_2018.index.strftime("%Y-%m-%d")

In [39]:
dummy_2016 = combined_data_pos_pls[combined_data_pos_pls["ticker"].isin(ticker_name["top_2015"])]
bool_dummy_2016 = dummy_2016[dummy_2016["year"] < 2017]
data_2016_returns = bool_dummy_2016[bool_dummy_2016["year"] > 2015]
dummy_2017 = combined_data_pos_pls[combined_data_pos_pls["ticker"].isin(ticker_name["top_2016"])]
bool_dummy_2017 = dummy_2017[dummy_2017["year"] < 2018]
data_2017_returns = bool_dummy_2017[bool_dummy_2017["year"] > 2016]
dummy_2018 = combined_data_pos_pls[combined_data_pos_pls["ticker"].isin(ticker_name["top_2017"])]
bool_dummy_2018 = dummy_2018[dummy_2018["year"] < 2019]
data_2018_returns = bool_dummy_2018[bool_dummy_2018["year"] > 2017]
dummy_2019 = combined_data_pos_pls[combined_data_pos_pls["ticker"].isin(ticker_name["top_2018"])]
bool_dummy_2019 = dummy_2019[dummy_2019["year"] < 2020]
data_2019_returns = bool_dummy_2019[bool_dummy_2019["year"] > 2018]

In [40]:
bottom_dummy_2016 = combined_data_pos_pls[combined_data_pos_pls["ticker"].isin(ticker_name["bot_2015"])]
bottom_bool_dummy_2016 = bottom_dummy_2016[bottom_dummy_2016["year"] < 2017]
bot_data_2016_returns = bottom_bool_dummy_2016[bottom_bool_dummy_2016["year"] > 2015]
bottom_dummy_2017 = combined_data_pos_pls[combined_data_pos_pls["ticker"].isin(ticker_name["bot_2016"])]
bottom_bool_dummy_2017 = bottom_dummy_2017[bottom_dummy_2017["year"] < 2018]
bot_data_2017_returns = bottom_bool_dummy_2017[bottom_bool_dummy_2017["year"] > 2016]
bottom_dummy_2018 = combined_data_pos_pls[combined_data_pos_pls["ticker"].isin(ticker_name["bot_2017"])]
bottom_bool_dummy_2018 = bottom_dummy_2018[bottom_dummy_2018["year"] < 2019]
bot_data_2018_returns = bottom_bool_dummy_2018[bottom_bool_dummy_2018["year"] > 2017]
bottom_dummy_2019 = combined_data_pos_pls[combined_data_pos_pls["ticker"].isin(ticker_name["bot_2018"])]
bottom_bool_dummy_2019 = bottom_dummy_2019[bottom_dummy_2019["year"] < 2020]
bot_data_2019_returns = bottom_bool_dummy_2019[bottom_bool_dummy_2019["year"] > 2018]

In [41]:
bot_first_2015 = data_bot_2015.T[data_bot_2015.T.index.isin(data_bot_2015.T.index[data_bot_2015.T.index.isin(bot_data_2016_returns["ticker"])])]
bot_first_2016 = data_bot_2016.T[data_bot_2016.T.index.isin(data_bot_2016.T.index[data_bot_2016.T.index.isin(bot_data_2017_returns["ticker"])])]
bot_first_2017 = data_bot_2017.T[data_bot_2017.T.index.isin(data_bot_2017.T.index[data_bot_2017.T.index.isin(bot_data_2018_returns["ticker"])])]
bot_first_2018 = data_bot_2018.T[data_bot_2018.T.index.isin(data_bot_2018.T.index[data_bot_2018.T.index.isin(bot_data_2019_returns["ticker"])])]

In [42]:
shift_value =  [-1,-4]
bottom_return_list_2016 = []
bottom_return_list_2017 = []
bottom_return_list_2018 = []
bottom_return_list_2019 = []

for i in shift_value:
    calculation_2016 = event_return(bot_first_2015.T, i, bot_data_2016_returns["date"], bot_data_2016_returns.groupby(["ticker","date"]).sum().index)
    calculation_2017 = event_return(bot_first_2016.T, i, bot_data_2017_returns["date"], bot_data_2017_returns.groupby(["ticker","date"]).sum().index)
    calculation_2018 = event_return(bot_first_2017.T, i, bot_data_2018_returns["date"], bot_data_2018_returns.groupby(["ticker","date"]).sum().index)
    calculation_2019 = event_return(bot_first_2018.T, i, bot_data_2019_returns["date"], bot_data_2019_returns.groupby(["ticker","date"]).sum().index)
    bottom_return_list_2016.append(calculation_2016)
    bottom_return_list_2017.append(calculation_2017)
    bottom_return_list_2018.append(calculation_2018)
    bottom_return_list_2019.append(calculation_2019)

In [66]:
bottom_strategy = pd.concat([(pd.Series(bottom_return_list_2016[0]) * 1 + pd.Series(bottom_return_list_2016[1]) * -1),\
          (pd.Series(bottom_return_list_2017[0]) * 1 + pd.Series(bottom_return_list_2017[1]) * -1),\
          (pd.Series(bottom_return_list_2018[0]) * 1 + pd.Series(bottom_return_list_2018[1]) * -1),\
          (pd.Series(bottom_return_list_2019[0]) * 1 + pd.Series(bottom_return_list_2019[1]) * -1)],axis =1)
bottom_strategy.columns = ["2016","2017","2018","2019"]

In [44]:
print(bottom_strategy.cumsum().apply(np.exp)["2016"].dropna().iloc[-1] - 1)
print(bottom_strategy.cumsum().apply(np.exp)["2017"].dropna().iloc[-1] - 1)
print(bottom_strategy.cumsum().apply(np.exp)["2018"].dropna().iloc[-1] - 1)
print(bottom_strategy.cumsum().apply(np.exp)["2019"].dropna().iloc[-1] - 1)

-0.12766990484102358
-0.1205041198676059
-0.22835955108705963
-0.010585634184374304


In [45]:
bottom_cumsum = pd.concat([pd.Series(bottom_strategy.cumsum().apply(np.exp)["2016"].dropna().iloc[-1]),\
pd.Series(bottom_strategy.cumsum().apply(np.exp)["2017"].dropna().iloc[-1]),\
pd.Series(bottom_strategy.cumsum().apply(np.exp)["2018"].dropna().iloc[-1]),\
pd.Series(bottom_strategy.cumsum().apply(np.exp)["2019"].dropna().iloc[-1])],axis = 1)
bottom_cumsum.columns = ["2016","2017","2018","2019"]

In [46]:
bottom_sharpe = pd.concat([pd.Series(sharpe(bottom_strategy["2016"])),pd.Series(sharpe(bottom_strategy["2017"])),\
                          pd.Series(sharpe(bottom_strategy["2018"])),pd.Series(sharpe(bottom_strategy["2019"]))],axis = 1)
bottom_sharpe.columns = ["2016","2017","2018","2019"]
bottom_sharpe

,2016,2017,2018,2019
0,-2.493,-2.484,-8.221,-0.130


# S&P 500

In [47]:
all_spx = getting_data((combined_data_pos_pls["ticker"].unique()).tolist(),dt.datetime(2015,1,1),dt.datetime(2019,12,29))["Close"]

C:\Users\fokta\anaconda3\lib\site-packages\pandas_datareader\base.py:272: SymbolWarning: Failed to read symbol: 'ILMN', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)


In [48]:
all_spx.index = all_spx.index.strftime("%Y-%m-%d")

In [49]:
spx_first = all_spx.T[all_spx.T.index.isin(all_spx.T.index[all_spx.T.index.isin(combined_data_pos_pls["ticker"])])]

In [50]:
spx_bool_dummy_2016 = combined_data_pos_pls[combined_data_pos_pls["year"] < 2017]
spx_2016_returns = spx_bool_dummy_2016[spx_bool_dummy_2016["year"] > 2015]
spx_bool_dummy_2017 = combined_data_pos_pls[combined_data_pos_pls["year"] < 2018]
spx_2017_returns = spx_bool_dummy_2017[spx_bool_dummy_2017["year"] > 2016]
spx_bool_dummy_2018 = combined_data_pos_pls[combined_data_pos_pls["year"] < 2019]
spx_2018_returns = spx_bool_dummy_2018[spx_bool_dummy_2018["year"] > 2017]
spx_bool_dummy_2019 = combined_data_pos_pls[combined_data_pos_pls["year"] < 2020]
spx_2019_returns = spx_bool_dummy_2019[spx_bool_dummy_2019["year"] > 2018]

In [51]:
shift_value =  [-1,-4]
spx_return_list_2016 = []
spx_return_list_2017 = []
spx_return_list_2018 = []
spx_return_list_2019 = []

for i in shift_value:
    calculation_2016 = event_return(spx_first.T, i, spx_2016_returns["date"], spx_2016_returns.groupby(["ticker","date"]).sum().index)
    calculation_2017 = event_return(spx_first.T, i, spx_2017_returns["date"], spx_2017_returns.groupby(["ticker","date"]).sum().index)
    calculation_2018 = event_return(spx_first.T, i, spx_2018_returns["date"], spx_2018_returns.groupby(["ticker","date"]).sum().index)
    calculation_2019 = event_return(spx_first.T, i, spx_2019_returns["date"], spx_2019_returns.groupby(["ticker","date"]).sum().index)
    spx_return_list_2016.append(calculation_2016)
    spx_return_list_2017.append(calculation_2017)
    spx_return_list_2018.append(calculation_2018)
    spx_return_list_2019.append(calculation_2019)

In [52]:
spx_strategy = pd.concat([(pd.Series(spx_return_list_2016[0]) * 1 + pd.Series(spx_return_list_2016[1]) * -1),\
          (pd.Series(spx_return_list_2017[0]) * 1 + pd.Series(spx_return_list_2017[1]) * -1),\
          (pd.Series(spx_return_list_2018[0]) * 1 + pd.Series(spx_return_list_2018[1]) * -1),\
          (pd.Series(spx_return_list_2019[0]) * 1 + pd.Series(spx_return_list_2019[1]) * -1)],axis =1)
spx_strategy.columns = ["2016","2017","2018","2019"]
spx_strategy

,2016,2017,2018,2019
0,0.026,-0.044,0.010,-0.001
1,-0.070,0.023,0.041,-0.030
2,-0.018,-0.033,0.065,0.049
3,-0.050,-0.014,0.021,0.007
4,-0.035,-0.022,-0.031,-0.015
...,...,...,...,...
753,NaN,-0.003,-0.013,NaN
754,NaN,0.002,0.048,NaN
755,NaN,0.001,0.007,NaN
756,NaN,-0.000,-0.001,NaN


In [53]:
print(spx_strategy.cumsum().apply(np.exp)["2016"].dropna().iloc[-1] -1)
print(spx_strategy.cumsum().apply(np.exp)["2017"].dropna().iloc[-1] -1)
print(spx_strategy.cumsum().apply(np.exp)["2018"].dropna().iloc[-1] -1)
print(spx_strategy.cumsum().apply(np.exp)["2019"].dropna().iloc[-1] -1)

-0.5856602897050658
0.7569551728543207
0.7964397724210572
14.021339367765766


In [54]:
spx_cumsum = pd.concat([pd.Series(spx_strategy.cumsum().apply(np.exp)["2016"].dropna().iloc[-1]),\
pd.Series(spx_strategy.cumsum().apply(np.exp)["2017"].dropna().iloc[-1]),\
pd.Series(spx_strategy.cumsum().apply(np.exp)["2018"].dropna().iloc[-1]),\
pd.Series(spx_strategy.cumsum().apply(np.exp)["2019"].dropna().iloc[-1])],axis = 1)
spx_cumsum.columns = ["2016","2017","2018","2019"]

In [55]:
spx_sharpe = pd.concat([pd.Series(sharpe(spx_strategy["2016"])),pd.Series(sharpe(spx_strategy["2017"])),\
                          pd.Series(sharpe(spx_strategy["2018"])),pd.Series(sharpe(spx_strategy["2019"]))],axis = 1)
spx_sharpe.columns = ["2016","2017","2018","2019"]
spx_sharpe

,2016,2017,2018,2019
0,-0.734,0.402,0.297,1.218


# Cumulative Return

In [60]:
cumulative_return_table = pd.concat([top_cumsum, bottom_cumsum, spx_cumsum]).reset_index(drop = True) -1
cumulative_return_table.index = ["top_30", "bottom_30", "spx"]
cumulative_return_table 

,2016,2017,2018,2019
top_30,-0.003,0.198,-0.018,0.127
bottom_30,-0.128,-0.121,-0.228,-0.011
spx,-0.586,0.757,0.796,14.021


# Sharpe Ratio

In [57]:
sharpe_table = pd.concat([top_sharpe, bottom_sharpe, spx_sharpe]).reset_index(drop = True)
sharpe_table.index = ["top_30", "bottom_30", "spx"]
sharpe_table

,2016,2017,2018,2019
top_30,-0.033,1.479,-0.117,0.682
bottom_30,-2.493,-2.484,-8.221,-0.130
spx,-0.734,0.402,0.297,1.218
